In [1]:
import pyvista as pv
import numpy as np
import matplotlib.pyplot as mt
from matplotlib.colors import ListedColormap
import vtk 
import vtkmodules 


In [2]:
# Converts a binary file of data to Vtk for the specific data sets of the 2006 IEE.
# fileSource: String - Path to the file to convert
# fileDestination: String - Path to the file to write  
# Return: Void
def binaryToVtk(fileSource, fileDestination):
    # Reader to get the data from the file. This describes the on-disk file
    # format so that VTK can parse it correctly.
    reader = vtk.vtkImageReader()
    reader.SetFileName(fileSource)
    # There are three dimensions in the file (X, Y, and Z)
    # Note that this file only stores the value for the X
    # component of the velocity, but it does this over the
    # whole 3D volume.
    reader.SetFileDimensionality(3)

    # There is one scalar field stored, and it is in big-endian floats
    reader.SetDataScalarTypeToFloat()
    reader.SetDataByteOrderToBigEndian()
    reader.SetNumberOfScalarComponents(1)

    # The first and last index of data values in X, Y, and Z
    reader.SetDataExtent(0, 749, 0, 374, 0, 99)

    # Picking an origin at zero, for no good reason
    reader.SetDataOrigin(0, 0, 0)

    # The data samples are 800 meters apart in X,Y,Z
    reader.SetDataSpacing(800, 800, 800)
    reader.Update()

    # Check if the file exists
    # Check if the file format is supported
    # Get the output data object
    output = reader.GetOutput()
    # Print image properties
    dimensions = output.GetDimensions()
    data_type = output.GetScalarTypeAsString()
    scalar_range = output.GetScalarRange()
    # Flip the image over in Y to make the coordinate system match
    # that of the data computation. The VTK image reader assumes
    # pixel (0,0) is at the upper-left, which is left-handed.
    reslice1 = vtk.vtkImageFlip()
    reslice1.SetInputData(reader.GetOutput())
    reslice1.SetFilteredAxis(1)
    reslice1.Update()
    # Write the data to a Structured Points file
    writer = vtk.vtkStructuredPointsWriter()
    writer.SetInputData(reslice1.GetOutput())
    writer.SetFileName(fileDestination)
    writer.SetFileTypeToBinary()
    writer.Write()

In [3]:
binaryToVtk("TS21z_X_R2_008000.bin", "FunctionX8000.vtk")
binaryToVtk("TS21z_Y_R2_008000.bin", "FunctionY8000.vtk")
binaryToVtk("TS21z_Z_R2_008000.bin", "FunctionZ8000.vtk")

2023-03-30 12:14:48.284 (   1.462s) [        FADC0740]    vtkImageReader2.cxx:578    ERR| vtkImageReader (0x559ea1edd390): Initialize: Could not open file TS21z_X_R2_008000.bin
ERROR:root:Initialize: Could not open file TS21z_X_R2_008000.bin
ERROR:root:Initialize: Could not open file TS21z_Y_R2_008000.bin
2023-03-30 12:14:49.360 (   2.539s) [        FADC0740]    vtkImageReader2.cxx:578    ERR| vtkImageReader (0x559ea1edd390): Initialize: Could not open file TS21z_Y_R2_008000.bin
ERROR:root:Initialize: Could not open file TS21z_Z_R2_008000.bin
2023-03-30 12:14:50.391 (   3.569s) [        FADC0740]    vtkImageReader2.cxx:578    ERR| vtkImageReader (0x559ea1edd390): Initialize: Could not open file TS21z_Z_R2_008000.bin


In [2]:
xfiles1to20 = [
    "TS21z_X_R2_000100",
    "TS21z_X_R2_001000",
    "TS21z_X_R2_002000",
    "TS21z_X_R2_003000",
    "TS21z_X_R2_004000",
    "TS21z_X_R2_005000",
    "TS21z_X_R2_006000",
    "TS21z_X_R2_007000",
    "TS21z_X_R2_008000",
    "TS21z_X_R2_009000",
    "TS21z_X_R2_010000",
    "TS21z_X_R2_011000",
    "TS21z_X_R2_012000",
    "TS21z_X_R2_013000",
    "TS21z_X_R2_014000",
    "TS21z_X_R2_015000",
    "TS21z_X_R2_016000",
    "TS21z_X_R2_017000",
    "TS21z_X_R2_018000",
    "TS21z_X_R2_019000",
    "TS21z_X_R2_020000"
            ]

In [5]:
# Create Vtk files from time stamps of 100 - 20000 for the x data.
for item in xfiles1to20:
    binaryToVtk("x1-20/"+item,"x1-20vtk/"+item+".vtk")

ERROR:root:Initialize: Could not open file x1-20/TS21z_X_R2_000100
2023-03-30 12:14:51.481 (   4.660s) [        FADC0740]    vtkImageReader2.cxx:578    ERR| vtkImageReader (0x559ea1ee9440): Initialize: Could not open file x1-20/TS21z_X_R2_000100
ERROR:root:Unable to open file: x1-20vtk/TS21z_X_R2_000100.vtk
2023-03-30 12:14:51.613 (   4.792s) [        FADC0740]      vtkDataWriter.cxx:187    ERR| vtkStructuredPointsWriter (0x559ea1ee3970): Unable to open file: x1-20vtk/TS21z_X_R2_000100.vtk
ERROR:root:Initialize: Could not open file x1-20/TS21z_X_R2_001000
2023-03-30 12:14:51.616 (   4.794s) [        FADC0740]    vtkImageReader2.cxx:578    ERR| vtkImageReader (0x559ea1ee9630): Initialize: Could not open file x1-20/TS21z_X_R2_001000
ERROR:root:Unable to open file: x1-20vtk/TS21z_X_R2_001000.vtk
2023-03-30 12:14:51.721 (   4.900s) [        FADC0740]      vtkDataWriter.cxx:187    ERR| vtkStructuredPointsWriter (0x559ea1eed610): Unable to open file: x1-20vtk/TS21z_X_R2_001000.vtk
ERROR:root

In [6]:
data = pv.read("../x1-20vtk/TS21z_X_R2_008000.vtk")
print(max(data.point_data["ImageFile"]))
print(min(data.point_data["ImageFile"]))

0.59702307
-0.6680263


In [7]:
data = pv.read("../x1-20vtk/TS21z_X_R2_000100.vtk")
plotter = pv.Plotter(off_screen=True) 
plotter.add_mesh(data, cmap="seismic", clim=[-0.6680263,0.59702307])

plotter.open_gif("x1-20.gif")

pts = data.points.copy()

nframe = 21
for item in xfiles1to20[1:]:
    data = pv.read("../x1-20vtk/"+item+".vtk")
    plotter.update_scalars(data.point_data["ImageFile"]) 
    plotter.write_frame()

plotter.close()

In [8]:
d = pv.read("../TS21VelocityMesh_VS_R2.vtk")
n = np.array(d)
plotter = pv.Plotter()
contours = d.contour(3,rng=(611,2000))
plotter.add_mesh(contours, cmap="gray")
plotter.show()

Widget(value="<iframe src='http://localhost:37013/index.html?ui=P_0x7fc9c0f5e590_0&reconnect=auto' style='widt…

In [9]:
plotter = pv.Plotter(off_screen=True) 
data2 = pv.read("../TS21VelocityMesh_VS_R2.vtk")

scale = 4
scale_matrix = np.diag([scale,scale,scale])
data2 = data2.scale((4,4,4))
data2 = data2.threshold(2500, invert=True)
plotter.add_mesh(data2, color="green")

data = pv.read("../x1-20vtk/TS21z_X_R2_008000.vtk")
mesh = plotter.add_mesh(data, cmap="seismic", opacity=0.5)
plotter.show()

Widget(value="<iframe src='http://localhost:43155/index.html?ui=P_0x7fb43021e290_6&reconnect=auto' style='widt…

### Gif Creation of wave motion as Iso Volumes

In [16]:
data = pv.read("../x1-20vtk/TS21z_X_R2_001000.vtk")
plotter = pv.Plotter(off_screen=True) 

iso1 = data.threshold(0.01)
iso2 = data.threshold(-0.01, invert=True)

mesh1 = plotter.add_mesh(iso1,lighting=False, cmap="seismic", opacity=0.5,clim=[-0.6680263,0.59702307])
mesh2 = plotter.add_mesh(iso2,lighting=False, cmap="seismic", opacity=0.5,clim=[-0.6680263,0.59702307])

plotter.open_gif("x1-20Iso.gif")

for item in xfiles1to20[2:]:
    plotter.clear_actors()
    data = pv.read("../x1-20vtk/"+item+".vtk")
    iso1 = data.threshold(0.01)
    iso2 = data.threshold(-0.01, invert=True)
    mesh1 = plotter.add_mesh(iso1,lighting=False, cmap="seismic", opacity=0.5,clim=[-0.6680263,0.59702307])
    mesh2 = plotter.add_mesh(iso2,lighting=False, cmap="seismic", opacity=0.5,clim=[-0.6680263,0.59702307])
    plotter.write_frame()

plotter.close()

### Gif creation of Waves with Land Features overlayed on top


In [14]:
plotter = pv.Plotter(off_screen=True) 
landFeatures = pv.read("../TS21VelocityMesh_VS_R2.vtk")

scale = 4
scale_matrix = np.diag([scale,scale,scale])
landFeatures = landFeatures.scale((4,4,4))

landFeatures = landFeatures.threshold(2500, invert=True)
plotter.add_mesh(landFeatures, color="green")


waves = pv.read("../x1-20vtk/TS21z_X_R2_000100.vtk")
mesh = plotter.add_mesh(waves, cmap="seismic", opacity=0.5, clim=[-0.6680263,0.59702307])

plotter.open_gif("x1-20OverLand.gif")

for item in xfiles1to20[1:]:
    plotter.clear_actors()
    waves = pv.read("../x1-20vtk/"+item+".vtk")
    plotter.add_mesh(landFeatures, color="green")
    plotter.add_mesh(waves, cmap="seismic", opacity=0.5, clim=[-0.6680263,0.59702307])
    plotter.write_frame()

plotter.close()
